## Utility Classes to support JSON generation

In [58]:
import json
class XaccJSONEncoder(json.JSONEncoder):
    """
    JSON encoder for NumPy arrays and complex numbers.
    This functions as the standard JSON Encoder but adds support
    for encoding:
        complex numbers z as lists [z.real, z.imag]
        numpy.ndarrays as nested lists.
    """
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, complex):
            return [obj.real, obj.imag]
        if hasattr(obj, "to_dict"):
            return obj.to_dict()
        return super().default(obj)

def clean_nones(value):
    """
    Recursively remove all None values from dictionaries and lists, and returns
    the result as a new dictionary or list.
    """
    if isinstance(value, list):
        return [clean_nones(x) for x in value if x is not None]
    elif isinstance(value, dict):
        return {
            key: clean_nones(val)
            for key, val in value.items()
            if val is not None
        }
    else:
        return value    
    
def json_serialize(obj):
    return json.loads(json.dumps(clean_nones(obj), cls=XaccJSONEncoder))

In [59]:
class ChannelKrausOp:
    '''
    Represents a noise channel in terms of Kraus operators: 
    Args:
        noise_qubits (List[String]): qubits that this operator acts on (If none, applying on gate qubits)
        matrix: the list of Kraus ops (as square matrices) representing a noise channel.
        Note: must satisfy the CPTP condition
    '''
    def __init__(self, matrix, noise_qubits = None):
        self.matrix = matrix
        self.noise_qubits = noise_qubits
    
    """
    Returns:
        dict: a (JSON) dictionary for a KrausOp
    """
    def to_dict(self):
        kraus = {
            "matrix": list(self.matrix),
            "noise_qubits": self.noise_qubits
        }
        return json_serialize(kraus)

In [60]:
class GateNoise:
    '''
    Represents noise ops (as a list of Kraus operators) associated with a quantum gate
    Args:
        name (string): gate name
        register_location (List[String]): gate qubits (as a list of register labels)
        noise_channels (List[KrausOp]): list of noise channels
    '''    
    def __init__(self, name, register_location, noise_kraus_ops):
        self.name = name
        self.register_location = register_location
        self.noise_kraus_ops = noise_kraus_ops
    
    def to_dict(self):
        gate_noise = {
            "gate_name": self.name,
            "register_location": self.register_location,
            "noise_channels": self.noise_kraus_ops
        }
        return json_serialize(gate_noise)

In [61]:
class ReadoutError:
    '''
    Represents the error in measuring qubits.
    We use two error probabilities: prob_meas0_prep1 (P(0|1)) and prob_meas1_prep0 (P(1|0))
    to capture the ReadoutError.
    Note: these values will be fed to the simulators, which may use different simulation methods. 
    Args:
        register_location (String): qubit label
        prob_meas0_prep1: Prob(0|1)
        prob_meas1_prep0: Prob(1|0)
    '''    
    def __init__(self, register_location, prob_meas0_prep1, prob_meas1_prep0):
        self.register_location = register_location
        self.prob_meas0_prep1 = prob_meas0_prep1
        self.prob_meas1_prep0 = prob_meas1_prep0

    def to_dict(self):
        gate_noise = {
            "register_location": self.register_location,
            "prob_meas0_prep1": self.prob_meas0_prep1,
            "prob_meas1_prep0": self.prob_meas1_prep0
        }
        return json_serialize(gate_noise)

In [62]:
class NoiseModel:
    '''
    Represents a noise model as a list of gate noise specifications
    Args:
        bit_order: MSB or LSB 
        gate_noise (List[GateNoise]): list of gate noise specifications (for Kraus ops look-up)
        readout_errors (List[ReadoutError]): list of readout errors
    '''    
    def __init__(self, gate_noise, bit_order = 'MSB', readout_errors = None):
        self.gate_noise = gate_noise
        self.bit_order = bit_order
        self.readout_errors = readout_errors
    def to_dict(self):
        noise_model = {
            "gate_noise": self.gate_noise,
            "bit_order": self.bit_order,
            "readout_errors": self.readout_errors
        }
        return json_serialize(noise_model)    

## Examples

### Example 1: single-qubit depolarization

In this example, we show how to devise a depolarization noise channel and generate a noise model JSON for that.

In [63]:
import numpy as np
# primative gates and basic operators
sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
sz = np.array([[1,0],[0,-1]])
s0 = np.eye(2)
# expand depolarizing channel in the Krauss basis
EDP = lambda l: np.array((np.sqrt(1-l)*s0, np.sqrt(l/3.0)*sx, np.sqrt(l/3.0)*sy, np.sqrt(l/3.0)*sz)) 
# Trace preserving condition. 
# Check it -> should return identity matrix
TP = lambda l: np.tensordot(EDP(l).swapaxes(1,2),EDP(l),axes=((0,2),(0,2))) 

In [64]:
# depolarizing rate
gamma = 0.01                                        
depol = EDP(gamma)

#### Data for validation

We compute analytically the density matrix for a 'noisy' X gate having this depolarizing channel.
We generate a simple noise model that only contains noise info for X gate.

In [65]:
# Create a gate noise instance for X gate on qubit "0"
# having that depolarizing channel.
xNoise = GateNoise("X", ["0"], [ChannelKrausOp(depol)])
# The noise model just contains a single entry for now.
noiseModel = NoiseModel([xNoise])

In [66]:
# This JSON can be used with XACC to initialize noisy simulation.
json.dumps(noiseModel.to_dict())

'{"gate_noise": [{"gate_name": "X", "register_location": ["0"], "noise_channels": [{"matrix": [[[[0.99498743710662, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.99498743710662, 0.0]]], [[[0.0, 0.0], [0.05773502691896258, 0.0]], [[0.05773502691896258, 0.0], [0.0, 0.0]]], [[[0.0, 0.0], [0.0, -0.05773502691896258]], [[0.0, 0.05773502691896258], [0.0, 0.0]]], [[[0.05773502691896258, 0.0], [0.0, 0.0]], [[0.0, 0.0], [-0.05773502691896258, 0.0]]]]}]}], "bit_order": "MSB"}'

Validation result:
Applying the depolarizing channel on |1><1| density matrix (result of a perfect X gate)

In [67]:
# rho = |1><1|
rho = np.array([[0,0],[0,1]])
result = np.zeros((2,2), dtype=complex)
for kraus in depol:
   result +=  kraus@rho@kraus.conj().T
print(result)

[[0.00666667+0.j 0.        +0.j]
 [0.        +0.j 0.99333333+0.j]]


### Example 2: Amplitude damping channel

A single-qubit amplitude damping channel is described by the following Kraus matrices:

A0 = $\begin{bmatrix}1 & 0 \\ 0  &  \sqrt{1-ampl}\end{bmatrix}$; 
A1 = $\begin{bmatrix} 0 & \sqrt{ampl} \\ 0  &  0\end{bmatrix}$ 

Test case: create an amplitude damping channel with high amplitude (e.g. 25%) associated with X gates.

Verify that the measure probability (no readout noise) matches expectation: 25% 0, 75% 1 (an ideal X gate produces 100% 1)

In [68]:
AD = lambda l: np.array([np.array([[1.0 , 0.0],[0.0, np.sqrt(1.0 - l)]], dtype = complex), np.array([[0.0, np.sqrt(l)], [0.0, 0.0]], dtype = complex)])
ad_ampl = 0.25
amplitude_damping = AD(0.25)
# Create a simple noise model that only contains an amplitude damping channel on X gate.
xNoise = GateNoise("X", ["0"], [ChannelKrausOp(amplitude_damping)])
noiseModel = NoiseModel([xNoise])
json.dumps(noiseModel.to_dict())

# Validate the simulation results: ~25% |0>, ~75% |1> measure distribution.

'{"gate_noise": [{"gate_name": "X", "register_location": ["0"], "noise_channels": [{"matrix": [[[[1.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.8660254037844386, 0.0]]], [[[0.0, 0.0], [0.5, 0.0]], [[0.0, 0.0], [0.0, 0.0]]]]}]}], "bit_order": "MSB"}'

## Multi-qubit Kraus channels

When specifying Kraus channels affecting multiple qubits, the 'MSB' and 'LSB' convention will help remove any ambiguity in the Kraus matrix representation.

This is important for unbalanced channels (e.g. the amount of IX depolarizing is different from that of XI).

Each simulator will have each own bit-order convention (for internal storage of the state-vector/density matrix); hence we need to validate the representation based on the effect on measurement results.

In the following example, we create a 2-q noise channel that only acts on the first qubit (Q0).
Depending on the LSB/MSB convention, the kron order will be different (when creating the Kraus matrices). 
We then feed the noise model (in both LSB/MSB formats) to the simulator and make sure that it will indeed only induce decoherent effect on that qubit (Q0).

In [69]:
# Create a depolarizing channel on 2-q Hilbert space but only acting
# on the first qubit (q0) according to a specific bit-ordering convention (LSB/MSB)
def single_qubit_depol_channel_2q(gamma, order = 'MSB'):
    sx = np.array([[0,1],[1,0]])
    sy = np.array([[0,-1j],[1j,0]])
    sz = np.array([[1,0],[0,-1]])
    s0 = np.eye(2)
    
    if order == 'MSB':
        # MSB: q1q0
        return np.array((np.sqrt(1-gamma)*np.kron(s0, s0), np.sqrt(gamma/3.0)*np.kron(s0, sx), np.sqrt(gamma/3.0)*np.kron(s0, sy), np.sqrt(gamma/3.0)*np.kron(s0, sz)))
    else:
        # LSB: q0q1
        # Swap the kron order:
        return np.array((np.sqrt(1-gamma)*np.kron(s0, s0), np.sqrt(gamma/3.0)*np.kron(sx, s0), np.sqrt(gamma/3.0)*np.kron(sy, s0), np.sqrt(gamma/3.0)*np.kron(sz, s0)))      

In [70]:
gamma = 0.01                                        
depol_msb = single_qubit_depol_channel_2q(gamma, 'MSB')
cxNoise_msb = GateNoise("CNOT", ["0", "1"], [ChannelKrausOp(depol_msb)])
noiseModel_msb = NoiseModel([cxNoise_msb], 'MSB')
noise_json_msb = json.dumps(noiseModel_msb.to_dict())
print(noise_json_msb)

{"gate_noise": [{"gate_name": "CNOT", "register_location": ["0", "1"], "noise_channels": [{"matrix": [[[[0.99498743710662, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.99498743710662, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.99498743710662, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.99498743710662, 0.0]]], [[[0.0, 0.0], [0.05773502691896258, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.05773502691896258, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.05773502691896258, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.05773502691896258, 0.0], [0.0, 0.0]]], [[[0.0, 0.0], [0.0, -0.05773502691896258], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.05773502691896258], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, -0.05773502691896258]], [[0.0, 0.0], [0.0, 0.0], [0.0, 0.05773502691896258], [0.0, 0.0]]], [[[0.05773502691896258, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [-0.05773502691896258, 0.0], [

In [71]:
gamma = 0.01                                        
depol_lsb = single_qubit_depol_channel_2q(gamma, 'LSB')
cxNoise_lsb = GateNoise("CNOT", ["0", "1"], [ChannelKrausOp(depol_lsb)])
noiseModel_lsb = NoiseModel([cxNoise_lsb], 'LSB')
noise_json_lsb = json.dumps(noiseModel_lsb.to_dict())
print(noise_json_lsb)

{"gate_noise": [{"gate_name": "CNOT", "register_location": ["0", "1"], "noise_channels": [{"matrix": [[[[0.99498743710662, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.99498743710662, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.99498743710662, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.99498743710662, 0.0]]], [[[0.0, 0.0], [0.0, 0.0], [0.05773502691896258, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.05773502691896258, 0.0]], [[0.05773502691896258, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.05773502691896258, 0.0], [0.0, 0.0], [0.0, 0.0]]], [[[0.0, 0.0], [0.0, 0.0], [0.0, -0.05773502691896258], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, -0.05773502691896258]], [[0.0, 0.05773502691896258], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.05773502691896258], [0.0, 0.0], [0.0, 0.0]]], [[[0.05773502691896258, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.05773502691896258, 0.0], [0

### Validation

The validation here is to run a circuit that only contains a CNOT gate on different noisy simulators.
The noise model is supplied in either LSB or MSB convention.
Let the simulator perform the noisy simulation then perform measurements.

There are two checks:

(1) The resulting density matrices must be equal when using the MSB or LSB JSON files. 

That is to say, the convention in which the noise model is specified doesn't change the semantics of the noise model (producing the same results)

(2) All the simulators (regardless of their internal MSB/LSB convention) must only show noisy results on qubit Q0.

i.e. checking that we map the noise model's LSB/MSB to the simulator's LSB/MSB correctly. 



## Validate Readout Errors


Specifying readout probabilities (no gate noises) and validate the readout distribution.

Example: P(1|0) = 0.1; P(0|1) = 0.2

- Run Identity circuit => get 90% 1, 10% 1

- Run X gate circuit => get 20% 0, 80% 1

In [72]:
prep_0_meas_1 = 0.1  
prep_1_meas_0 = 0.2                                      
# Readout error on qubit 0
ro_error = ReadoutError("0", prep_1_meas_0, prep_0_meas_1)
# No gate noises, just readout errors for validation.
noiseModel = NoiseModel([], 'MSB', [ro_error])
noise_json = json.dumps(noiseModel.to_dict())
print(noise_json)

{"gate_noise": [], "bit_order": "MSB", "readout_errors": [{"register_location": "0", "prob_meas0_prep1": 0.2, "prob_meas1_prep0": 0.1}]}
